In [2]:
require 'torch'
require 'nn'
require 'paths'
-- require 'cunn'
mnist = require 'mnist'

fullset = mnist.traindataset()
testset = mnist.testdataset()

for i=1,fullset.size do
    if fullset.label[i] == 0 then
        fullset.label[i] = 10
    end
end

dataset = {}

for i=1,fullset.size do
    dataset[i] = {fullset.data[i]:resize(784):double(), fullset.label[i]}
end

function dataset:size() 
    return fullset.size
end

mlp = nn.Sequential()
mlp:add(nn.Linear(784,400))
mlp:add(nn.Sigmoid())
-- mlp:add(nn.Linear(400,200))
mlp:add(nn.Linear(400,400))
mlp:add(nn.Sigmoid())
-- mlp:add(nn.Linear(400,10))
mlp:add(nn.Linear(400,200))
mlp:add(nn.Sigmoid())
mlp:add(nn.Linear(200,10))
mlp:add(nn.LogSoftMax())
-- mlp:cuda()
-- criterion = nn.ClassNLLCriterion()
criterion = nn.CrossEntropyCriterion()
-- criterion:cuda()
trainer = nn.StochasticGradient(mlp,criterion)
trainer.learningRate = 0.001
trainer.maxIteration = 1 -- just do 1 epochs of training.
-- trainer.help
-- trainer:cuda()

function train(model)
    model:train(dataset)
end

-- real magic of regularizatin will start here
function regularize(modelmodule, lambda)
--     local lambda = 5
    local parameters, _ = modelmodule:parameters()
    local mul_factor = lambda/dataset.size()
--     print(table.getn(parameters))
--     print(parameters[1]) -- before update --confirm the regularization is working
    for j= 1, #parameters do
        local add_factor = mul_factor*torch.pow(parameters[j],2)
        parameters[j]:add(add_factor)
    end
--     print(parameters[1]) --after update --used for confirm the regularization is working
end
-- magic of regularization ends here

epochs = 5
print("Model started training with regularization")
for i= 1, epochs do 
    train(trainer)
--     regularize(trainer.module, 15)
--     local parameters, _ = trainer.module:parameters()
--     print(parameters[1])    --is reflected in network?--these 2 lines are just to confirm the regularization is working
end
print("Model with reg trained")
torch.save("mnist_reg.t7",mlp)


mlp = torch.load('mnist_reg.t7')
for i=1,testset.size do
    if testset.label[i] == 0 then
        testset.label[i] = 10
    end
end   
    
correct = 0
for i=1,testset.size do
--     itorch.image(testset.data[i])
    local predict = mlp:forward(testset.data[i]:resize(784):double())
    local max = torch.max(predict:exp())    
    for j=1,10 do        
        if predict[j] == max then
--             print(j)
            if j == testset.label[i] then
                correct = correct + 1
            end            
        end
    end            
end

accuracy = (correct/testset.size)*100
print(accuracy)

Model started training with regularization	


# StochasticGradient: training	


# current error = 1.7320756756745	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 1.7320756756745	


# StochasticGradient: training	


# current error = 0.59135790558379	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.59135790558379	


...

# StochasticGradient: training	


# current error = 0.42216908373582	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.42216908373582	


# StochasticGradient: training	


# current error = 0.41538041541448	
# StochasticGradient: you have reached the maximum number of iterations	
# training error = 0.41538041541448	
Model with reg trained	


88.67	
